In [4]:
import json
from copy import deepcopy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use("ggplot")

from Utils.dataset import *
from Utils.utils import *
from Models.BertClf import * 
from Models.LstmClf import *
from Models.ElectraClf import *
from Models.ConvClf import *

device = torch.device("cuda:1" if torch.cuda.is_available() else torch.device("cpu"))
# device = torch.device("cpu")
torch.cuda.set_device(device) # change allocation of current GPU
print(f'training device: {device, torch.cuda.get_device_name()}')

training device: (device(type='cuda', index=1), 'GeForce GTX 1080 Ti')


In [3]:
# Define signature
"""학습 잘시킨 놈으로 다가 BILSTM, CNN, BERT, ELECTRA가져와"""
signature_list = ["jh_BILSTM_6M_1D_11H_7M", 
                  "jh_CNN_5M_31D_19H_2M", 
                  "jh_BERT_5M_31D_18H_42M", 
                  "jh_ELECTRA_5M_31D_18H_23M"]

In [5]:
# Load options
def load_opt(signature):
    parser = argparse.ArgumentParser()
    #     opt = parser.parse_args() # in .py env
    opt, _ = parser.parse_known_args() # in .ipynb env
    with open('./Saved_models/' + signature + '_opt.txt', 'r') as f:
        opt.__dict__ = json.load(f)
    return opt

opt_list = []
for sign in signature_list:
    opt_list.append(load_opt(sign))
opt_list

[Namespace(aug=0, author='jh', batch_size=1024, data_path='./Dataset', dropout=0.5, embedding_dim=256, eps=1e-08, freeze_pretrained=0, gpu=0, hidden_dim=768, kernel_depth=500, kernel_sizes=[3, 4, 5], lr_clf=0.001, lr_pretrained=1e-05, max_epoch=40, max_len=50, model='BILSTM', num_layer=2, save=1, save_model_path='./Saved_models', save_submission_path='./Submissions', sent_embedding=0, signature='jh_BILSTM_6M_1D_11H_7M', split_ratio=1, weight_decay=0.0005),
 Namespace(aug=0, author='jh', batch_size=1024, data_path='./Dataset', dropout=0.5, embedding_dim=256, eps=1e-08, freeze_pretrained=0, gpu=0, hidden_dim=768, kernel_depth=500, kernel_sizes=[3, 4, 5], lr_clf=0.0001, lr_pretrained=1e-05, max_epoch=60, max_len=50, model='CNN', num_layer=2, save=1, save_model_path='./Saved_models', save_submission_path='./Submissions', sent_embedding=0, signature='jh_CNN_5M_31D_19H_2M', split_ratio=1, weight_decay=0.0005),
 Namespace(aug=0, author='jh', batch_size=16, data_path='./Dataset', dropout=0.5, 

In [17]:
submission_templete_df = pd.read_csv(opt_list[0].data_path +'/sample_sub.csv') # a sample submission file in the correct format

_, valid_dataloader_bert, test_dataloader_bert = data_load(opt_list[0])
_, valid_dataloader_electra, test_dataloader_electra = data_load(opt_list[3])

Tokenizing data...
Apply the BertTokenizer...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


train_X_ids_tsr.shape: torch.Size([7805, 50])
train_X_masks_tsr.shape: torch.Size([7805, 50])
Tokenizing data...
Apply the BertTokenizer...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


valid_X_ids_tsr.shape: torch.Size([1748, 50])
valid_X_masks_tsr.shape: torch.Size([1748, 50])
Tokenizing data...
Apply the BertTokenizer...


...

train_X_ids_tsr.shape: torch.Size([7805, 50])
train_X_masks_tsr.shape: torch.Size([7805, 50])
Tokenizing data...
Apply the ElectraTokenizer...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


valid_X_ids_tsr.shape: torch.Size([1748, 50])
valid_X_masks_tsr.shape: torch.Size([1748, 50])
Tokenizing data...
Apply the ElectraTokenizer...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


test_X_ids_tsr.shape: torch.Size([4311, 50])
test_X_masks_tsr.shape: torch.Size([4311, 50])
num of train_loader: 7805
num of valid_loader: 1748
num of test_loader: 4311


In [10]:
model_bilstm = LstmClassifier(opt_list[0])
model_bilstm.to(device)
model_save_path = str(opt_list[0].save_model_path) + "/" + opt_list[0].signature +'.model'
model_bilstm.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [11]:
model_cnn = ConvClassifier(opt_list[1])
model_cnn.to(device)
model_save_path = str(opt_list[1].save_model_path) + "/" + opt_list[1].signature +'.model'
model_cnn.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [12]:
model_bert = BertClassifier(opt_list[2])
model_bert.to(device)
model_save_path = str(opt_list[2].save_model_path) + "/" + opt_list[2].signature +'.model'
model_bert.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [13]:
model_electra = ElectraClassifier(opt_list[3])
model_electra.to(device)
model_save_path = str(opt_list[3].save_model_path) + "/" + opt_list[3].signature +'.model'
model_electra.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [14]:
def make_preds(model, opt, device, test_dataloader, flag="valid"):
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()
    all_logits = []
    all_labels = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch 
        if flag == "valid":
            b_ids_tsr, b_masks_tsr, b_labels_tsr = tuple(tsr.to(device) for tsr in batch)[:3]
        else:
            b_ids_tsr, b_masks_tsr = tuple(tsr.to(device) for tsr in batch)[:2]

        # Compute logits
        with torch.no_grad():
            if opt.model in ["BILSTM", "CNN"]:
                logits = model(b_ids_tsr)
            else:
                logits = model(b_ids_tsr, b_masks_tsr)
        all_logits.append(logits)
        if flag == "valid":
            all_labels.append(b_labels_tsr)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)
    if flag == "valid":
        all_labels = torch.cat(all_labels, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()
    return probs, all_labels

In [19]:
bilstm_valid_probs, labels1 = make_preds(model_bilstm, opt_list[0], device, valid_dataloader_bert)
bilstm_test_probs, _ = make_preds(model_bilstm, opt_list[0], device, test_dataloader_bert, flag="test")
print(f"bilstm_valid_probs: {bilstm_valid_probs.shape}, bilstm_test_probs: {bilstm_test_probs.shape}, labels: {labels1.shape}")

bilstm_valid_probs: (1748, 5), bilstm_test_probs: (4311, 5), labels: torch.Size([1748])


In [20]:
cnn_valid_probs, labels2 = make_preds(model_cnn, opt_list[1], device, valid_dataloader_bert)
cnn_test_probs, _ = make_preds(model_cnn, opt_list[1], device, test_dataloader_bert, flag="test")
print(f"cnn_valid_probs: {cnn_valid_probs.shape}, cnn_test_probs: {cnn_test_probs.shape}, labels: {labels2.shape}")

cnn_valid_probs: (1748, 5), cnn_test_probs: (4311, 5), labels: torch.Size([1748])


In [21]:
bert_valid_probs, labels3 = make_preds(model_bert, opt_list[2], device, valid_dataloader_bert)
bert_test_probs, _ = make_preds(model_bert, opt_list[2], device, test_dataloader_bert, flag="test")
print(f"bert_valid_probs: {bert_valid_probs.shape}, bert_test_probs: {bert_test_probs.shape}, labels: {labels3.shape}")

bert_valid_probs: (1748, 5), bert_test_probs: (4311, 5), labels: torch.Size([1748])


In [22]:
electra_valid_probs, labels4 = make_preds(model_electra, opt_list[3], device, valid_dataloader_electra)
electra_test_probs, _ = make_preds(model_electra, opt_list[3], device, test_dataloader_electra, flag="test")
print(f"electra_valid_probs: {electra_valid_probs.shape}, electra_test_probs: {electra_test_probs.shape}, labels: {labels4.shape}")

electra_valid_probs: (1748, 5), electra_test_probs: (4311, 5), labels: torch.Size([1748])


In [28]:
labels1 = labels1.detach().cpu().numpy()
labels2 = labels2.detach().cpu().numpy()
labels3 = labels3.detach().cpu().numpy()
labels4 = labels4.detach().cpu().numpy()
if sum(labels1 == labels2) == sum(labels1 == labels3) & sum(labels1 == labels2) == sum(labels1 == labels4):
    print("그래 진행시켜")
labels = labels1

그래 진행시켜


In [47]:
sum(bilstm_valid_preds == labels) / len(labels) # bilstm valid acc

0.5617848970251716

In [48]:
sum(cnn_valid_preds == labels) / len(labels) # cnn valid acc

0.6384439359267735

In [51]:
sum(bert_valid_preds == labels) / len(labels) # bert valid acc

0.7551487414187643

In [52]:
sum(electra_valid_preds == labels) / len(labels) # electra valid acc

0.7402745995423341

In [33]:
# HitDifferenceRatio on valid dataset
bilstm_valid_preds = np.argmax(bilstm_valid_probs, axis=1)
cnn_valid_preds = np.argmax(cnn_valid_probs, axis=1)
bert_valid_preds = np.argmax(bert_valid_probs, axis=1)
electra_valid_preds = np.argmax(electra_valid_probs, axis=1)

# (Electra, Bert), (Electra, Bilstm), (Electra, Cnn)
def calDHR(preds1, preds2, labels=labels):    
    union_arr = np.where((labels == preds1) | (labels == preds2))
    intersection_arr = (np.where((labels == preds1) & (labels == preds2)))
    sizeOfunion_int = len(union_arr[0])
    sizeOfintersection_int = len(intersection_arr[0])
    HDR = (sizeOfunion_int - sizeOfintersection_int) / sizeOfunion_int
#     HDR # 높다는건 교집합이 작다 즉 서로 알려줄게 많다.
    return HDR

In [34]:
calDHR(electra_valid_preds, bert_valid_preds) # electra - bert

0.2229775662814412

In [35]:
calDHR(electra_valid_preds, bilstm_valid_preds) # electra - bilstm

0.4249134948096886

In [36]:
calDHR(electra_valid_preds, cnn_valid_preds) # electra - cnn

0.3344851416724257

In [37]:
calDHR(bert_valid_preds, bilstm_valid_preds) # bert - bilstm

0.40360610263522884

In [38]:
calDHR(bert_valid_preds, cnn_valid_preds) # bert - cnn

0.3000697836706211

In [39]:
calDHR(bilstm_valid_preds, cnn_valid_preds) # bilstm - cnn

0.38117283950617287

In [ ]:
def ensemble_preds(compare_valid_probs):
    # Average on valid dataset
    avg_preds = (electra_valid_probs + compare_valid_probs) * 0.5
    avg_preds = np.argmax(avg_preds, axis=1)
    print("avg valid acc:", sum(avg_preds == labels) / len(labels)) # avg valid acc

    # Confidence on valid dataset
    conf_preds = []
    for i in range(len(labels)):
        if np.max(electra_valid_probs[i]) > np.max(compare_valid_probs[i]):
            conf_preds.append(electra_valid_preds[i])
        else:
            conf_preds.append(compare_valid_probs[i])
    conf_preds = (np.array(conf_preds))
    print("conf valid acc:", sum(conf_preds == labels) / len(labels)) # conf valid acc

    # Standard Deviation on valid dataset
    std_preds = []
    for i in range(len(labels)):
        if np.std(electra_valid_probs[i]) > np.std(compare_valid_probs[i]):
            std_preds.append(electra_valid_preds[i])
        else:
            std_preds.append(compare_valid_probs[i])
    std_preds = (np.array(std_preds))
    print("std valid acc:", sum(std_preds == labels) / len(labels)) # std valid acc

    # top 2 on Valid dataset
    # trial_1
    # print(lstm_logits.shape)
    # print(electra_logits.shape)
    logits = []
    for i in range(electra_valid_probs.shape[0]):
        # print(lstm_logits[i].shape, lstm_logits[i])
        lstm_ls = compare_valid_probs[i].tolist()
        lstm_ls.sort(reverse=True)
        lstm_diff = lstm_ls[0] - lstm_ls[1]
        electra_ls = electra_logits[i].tolist()
        electra_ls.sort(reverse=True)
        electra_diff = electra_ls[0] - electra_ls[1]
        if lstm_diff>electra_diff:
            logits.append(lstm_logits[i].tolist())
        else:
            logits.append(electra_logits[i].tolist())
        # print(logits)
    logits = torch.Tensor(logits)
    all_logits.append(logits)



In [81]:
# Average on valid dataset
avg_preds = (electra_valid_probs + bilstm_valid_probs) * 0.5
avg_preds = np.argmax(avg_preds, axis=1)
print("avg valid acc:", sum(avg_preds == labels) / len(labels)) # avg valid acc

# Confidence on valid dataset
conf_preds = []
for i in range(len(labels)):
    if np.max(electra_valid_probs[i]) > np.max(bilstm_valid_probs[i]):
        conf_preds.append(electra_valid_preds[i])
    else:
        conf_preds.append(bilstm_valid_preds[i])
conf_preds = (np.array(conf_preds))
print("conf valid acc:", sum(conf_preds == labels) / len(labels)) # conf valid acc

# Standard Deviation on valid dataset
std_preds = []
for i in range(len(labels)):
    if np.std(electra_valid_probs[i]) > np.std(bilstm_valid_probs[i]):
        std_preds.append(electra_valid_preds[i])
    else:
        std_preds.append(bilstm_valid_preds[i])
std_preds = (np.array(std_preds))
print("std valid acc:", sum(std_preds == labels) / len(labels)) # std valid acc

avg valid acc: 0.7511441647597255
conf valid acc: 0.7408466819221968
std valid acc: 0.7425629290617849


In [ ]:
# Average on test dataset
avg_preds = (electra_test_probs + bilstm_test_probs) * 0.5
avg_preds = np.argmax(avg_preds, axis=1)
print("avg valid acc:", sum(avg_preds == labels) / len(labels)) # avg test acc

# Confidence on test dataset
conf_preds = []
for i in range(len(labels)):
    if np.max(electra_test_probs[i]) > np.max(bilstm_test_probs[i]):
        conf_preds.append(electra_test_preds[i])
    else:
        conf_preds.append(bilstm_test_preds[i])
conf_preds = (np.array(conf_preds))
print("conf valid acc:", sum(conf_preds == labels) / len(labels)) # conf test acc

# Standard Deviation on test dataset
std_preds = []
for i in range(len(labels)):
    if np.std(electra_test_probs[i]) > np.std(bilstm_test_probs[i]):
        std_preds.append(electra_test_preds[i])
    else:
        std_preds.append(bilstm_test_preds[i])
std_preds = (np.array(std_preds))
print("std valid acc:", sum(std_preds == labels) / len(labels)) # std test acc

# Choice
# submission_templete_df.Category = avg_preds
# submission_templete_df.to_csv('./Submissions/avg_electra_bilstm_preds.csv', index=False)
# submission_templete_df.Category = conf_preds
# submission_templete_df.to_csv('./Submissions/conf_electra_bilstm_preds.csv', index=False)
# submission_templete_df.Category = std_preds
# submission_templete_df.to_csv('./Submissions/std_electra_bilstm_preds.csv', index=False)

In [83]:
# Average on valid dataset
avg_preds = (electra_valid_probs + bert_valid_probs) * 0.5
avg_preds = np.argmax(avg_preds, axis=1)
print("avg valid acc:", sum(avg_preds == labels) / len(labels)) # avg valid acc

# Confidence on valid dataset
conf_preds = []
for i in range(len(labels)):
    if np.max(electra_valid_probs[i]) > np.max(bert_valid_probs[i]):
        conf_preds.append(electra_valid_preds[i])
    else:
        conf_preds.append(bert_valid_preds[i])
conf_preds = (np.array(conf_preds))
print("conf valid acc:", sum(conf_preds == labels) / len(labels)) # conf valid acc

# Standard Deviation on valid dataset
std_preds = []
for i in range(len(labels)):
    if np.std(electra_valid_probs[i]) > np.std(bert_valid_probs[i]):
        std_preds.append(electra_valid_preds[i])
    else:
        std_preds.append(bert_valid_preds[i])
std_preds = (np.array(std_preds))
print("std valid acc:", sum(std_preds == labels) / len(labels)) # std valid acc

avg valid acc: 0.7717391304347826
conf valid acc: 0.7711670480549199
std valid acc: 0.7705949656750573


In [84]:
# Average on test dataset
avg_preds = (electra_test_probs + bert_test_probs) * 0.5
avg_preds = np.argmax(avg_preds, axis=1)
print("avg valid acc:", sum(avg_preds == labels) / len(labels)) # avg test acc

# Confidence on test dataset
conf_preds = []
for i in range(len(labels)):
    if np.max(electra_test_probs[i]) > np.max(bert_test_probs[i]):
        conf_preds.append(electra_test_preds[i])
    else:
        conf_preds.append(bert_test_preds[i])
conf_preds = (np.array(conf_preds))
print("conf valid acc:", sum(conf_preds == labels) / len(labels)) # conf test acc

# Standard Deviation on test dataset
std_preds = []
for i in range(len(labels)):
    if np.std(electra_test_probs[i]) > np.std(bert_test_probs[i]):
        std_preds.append(electra_test_preds[i])
    else:
        std_preds.append(bert_test_preds[i])
std_preds = (np.array(std_preds))
print("std valid acc:", sum(std_preds == labels) / len(labels)) # std test acc

# Choice
# submission_templete_df.Category = avg_preds
# submission_templete_df.to_csv('./Submissions/avg_electra_bert_preds.csv', index=False)
# submission_templete_df.Category = conf_preds
# submission_templete_df.to_csv('./Submissions/conf_electra_bert_preds.csv', index=False)
# submission_templete_df.Category = std_preds
# submission_templete_df.to_csv('./Submissions/std_electra_bert_preds.csv', index=False)

array([3, 2, 1, ..., 3, 4, 2])

In [78]:
# Average on valid dataset
avg_preds = (bilstm_valid_probs + cnn_valid_probs) * 0.5
avg_preds = np.argmax(avg_preds, axis=1)
print("avg valid acc:", sum(avg_preds == labels) / len(labels)) # avg valid acc

# Confidence on valid dataset
conf_preds = []
for i in range(len(labels)):
    if np.max(bilstm_valid_probs[i]) > np.max(cnn_valid_probs[i]):
        conf_preds.append(bilstm_valid_preds[i])
    else:
        conf_preds.append(cnn_valid_preds[i])
conf_preds = (np.array(conf_preds))
print("conf valid acc:", sum(conf_preds == labels) / len(labels)) # conf valid acc

# Standard Deviation on valid dataset
std_preds = []
for i in range(len(labels)):
    if np.std(bilstm_valid_probs[i]) > np.std(cnn_valid_probs[i]):
        std_preds.append(bilstm_valid_preds[i])
    else:
        std_preds.append(cnn_valid_preds[i])
std_preds = (np.array(std_preds))
print("std valid acc:", sum(std_preds == labels) / len(labels)) # std valid acc

avg valid acc: 0.6361556064073226
conf valid acc: 0.6224256292906178
std valid acc: 0.6189931350114416


In [ ]:
# Average on test dataset
avg_preds = (bilstm_test_probs + cnn_test_probs) * 0.5
avg_preds = np.argmax(avg_preds, axis=1)
print("avg valid acc:", sum(avg_preds == labels) / len(labels)) # avg test acc

# Confidence on test dataset
conf_preds = []
for i in range(len(labels)):
    if np.max(bilstm_test_probs[i]) > np.max(cnn_test_probs[i]):
        conf_preds.append(bilstm_test_preds[i])
    else:
        conf_preds.append(cnn_test_preds[i])
conf_preds = (np.array(conf_preds))
print("conf valid acc:", sum(conf_preds == labels) / len(labels)) # conf test acc

# Standard Deviation on test dataset
std_preds = []
for i in range(len(labels)):
    if np.std(bilstm_test_probs[i]) > np.std(cnn_test_probs[i]):
        std_preds.append(bilstm_test_preds[i])
    else:
        std_preds.append(cnn_test_preds[i])
std_preds = (np.array(std_preds))
print("std valid acc:", sum(std_preds == labels) / len(labels)) # std test acc

# Choice
# submission_templete_df.Category = avg_preds
# submission_templete_df.to_csv('./Submissions/avg_bilstm_cnn_preds.csv', index=False)
# submission_templete_df.Category = conf_preds
# submission_templete_df.to_csv('./Submissions/conf_bilstm_cnn_preds.csv', index=False)
# submission_templete_df.Category = std_preds
# submission_templete_df.to_csv('./Submissions/std_bilstm_cnn_preds.csv', index=False)